# Overview
In this notebook we will consider vibrational strong coupling between a cavity and the HF molecule where the cavity
is tuned to the fundamental transition of HF within the Harmonic Oscillator approximation.  

We will deploy two different Hamiltonians: The minimal coupling Hamiltonian (`` p dot A ''), and the Pauli Fierz Hamiltonian.

The form of these coupled Hamiltonians are
\begin{equation}
    \hat{H}_{\rm p \cdot A} = \frac{\hat{p}^2}{2m} + V(\hat{x})  + \hbar \omega_{\rm cav} \hat{b}^{\dagger} \hat{b} -  \frac{z}{m} \hat{p} \cdot  {\bf A}_0 ( \hat{b}^{\dagger} + \hat{b}) + \frac{z^2}{2m}  {\bf A}_0^2 ( \hat{b}^{\dagger} + \hat{b})^2,
\end{equation}
for the minimal coupling Hamiltonian, and 
\begin{equation}
    \hat{H}_{\rm PF} = \frac{\hat{p}^2}{2m}+ V(\hat{x}) + \hbar \omega_{\rm cav} \hat{b}^{\dagger} \hat{b} - \omega_{\rm cav} \hat{\mu} \cdot {\bf A}_0 ( \hat{b}^{\dagger} + \hat{b}) +\frac{\omega_{{\rm cav}}}{\hbar} ( \hat{\mu} \cdot {\bf A}_0)^2
\end{equation}
for the Pauli-Fierz Hamiltonian.

Here, we will set the mass $m$ to be the reduced mass of the HF molecule,
\begin{equation}
m = \frac{m_F \cdot m_H}{m_F + m_H}
\end{equation}
and we will define the potential $V(x)$ to be the Harmonic potential,
\begin{equation}
V(x) = \frac{1}{2} k x^2.
\end{equation}

# Unit system
We will work in atomic units.  In atomic units, so the mass of H and F, the Harmonic force constant, the cavity frequency, the charge
associated with the dipole operator, and the vector potential must all be expressed in atomic units.  We will consider an arbitrary range of values for the vector potential, but all other values will be fixed as follows:

| Quantity    | Value in Atomic Units |
| -------- | ------- |
| mass of F  | 34616.6811    |
| mass of H  | 1837.4731    |
| Force Constant  of HF* |  0.6377 |
| Equilibrium bondlength* | 1.7310 |
| Charge*   | -0.4688 |
| Cavity frequency* | 0.01911


The force constant, charge, and cavity frequency are all quantities that we have derived from some calculations.  
We estimated the force constant using quantum chemistry calculations, specifically using numerical derivatives of CCSD(T)/cc-pVTZ 
energies about the equilibrium geometry located by a geometry optimization at the same level.  Similarly, we computed the RHF/cc-pVTZ dipole moment at this optimized geometry and derived the effective charge as the dipole moment divided by the equilibrium bond length.  The
cavity frequency is matched to the fundamental frequency of HF, which is derived from the Harmonic approximation as follows:
\begin{equation}
\omega_{\rm cav} = \omega_{\rm f} = \sqrt{ \frac{k}{m} }
\end{equation}
where $k$ is the force constant and $m$ is the reduced mass and $\omega_{\rm f}$ indicates the fundamental frequency of the HF molecule.

In [11]:
import numpy as np
from numpy import linalg as la

# Force constant of HF in atomic units as computed using centered finite differences at the CCSD(T)/cc-pVTZ level 
# about an equilibrium bondlength of 1.73106 atomic units (0.9160 Angstroms )
k_au = 0.6377016832933821

# reduced mass of HF
mF_au = 34616.6811
mH_au = 1837.4731

mu_au = mF_au * mH_au / (mF_au + mH_au)

# fundamental frequency of HF in atomic units
omega_au = np.sqrt( k_au / mu_au )

# set cavity frequency equal to fundamental
omega_cav  = omega_au 

print(F' Cavity Frequency in Atomic Units is {omega_cav}')



 Cavity Frequency in Atomic Units is 0.019117411406998054


With these basic parameters having been computed or assigned, we will write functions that compute matrix
elements of each term in the two Hamiltonians we are considering.  These functions will all have a common structure that 
they will take arguments that specify the matter and photonic basis functions for the bra and ket, and any relevant parameters, and will return the value of the matrix element.  We will reiterate the relevant matrix element above each block for clarity.

The function `compute_matter_matrix_element(q_m, q_p, r_m, r_p, k, mu)` will compute the pure matter matrix elements
\begin{equation}
\langle q_m | \langle q_p | \frac{\hat{p}^2}{2m} + \frac{1}{2} k \hat{x}^2| r_m \rangle | r_p \rangle = \hbar \omega_{\rm f} (r_m + \frac{1}{2}) \delta_{q_m, r_m} \delta_{q_p, r_p}
\end{equation}
which is the same for both the minimal coupling and Pauli-Fierz Hamiltonians.

In [12]:
def compute_matter_matrix_element(q_m, q_p, r_m, r_p, k, m):
    """ 
    Arguments
    ---------
    q_m : int
        matter bra index

    q_p : int
        photonic bra index

    r_m : int
        matter ket index

    r_p : int
        photon ket index

    k : float
        force constant

    mu : float
        reduced mass

    Returns
    -------

    E_matter : float
        Matter matrix element 
    """
    # compute the magnitude of the matrix element
    E_matter = np.sqrt( k / m) * (r_m + 1/2)
    
    # returns only if diagonal in matter and photon bra and kets
    return E_matter * (q_m == r_m) * (q_p == r_p)


The function `compute_photon_matrix_element(q_m, q_p, r_m, r_p, omega)` will compute the pure photon matrix elements
\begin{equation}
\langle q_m | \langle q_p | \hbar \omega_{\rm cav} (\hat{b}^{\dagger} \hat{b} + \frac{1}{2} ) | r_m \rangle | r_p \rangle = \hbar \omega_{\rm cav} (r_p + \frac{1}{2}) \delta_{q_m, r_m} \delta_{q_p, r_p}
\end{equation}
which is the same for both the minimal coupling and Pauli-Fierz Hamiltonians.

In [13]:
def compute_photon_matrix_element(q_m, q_p, r_m, r_p, omega):
    """ 
    Arguments
    ---------
    q_m : int
        matter bra index

    q_p : int
        photonic bra index

    r_m : int
        matter ket index

    r_p : int
        photon ket index

    omega : float
        cavity frequency

    Returns
    -------

    E_photon : float
        Photon matrix element 
    """
    # compute the magnitude of the matrix element
    E_photon = omega * (r_p + 1/2)
    
    # returns only if diagonal in matter and photon bra and kets
    return E_photon * (q_m == r_m) * (q_p == r_p)

The function `compute_diamagnetic_element_p_dot_A(q_m, q_p, r_m, r_p, z_charge, A0, m)` computes the matrix element of 

\begin{align}
& \langle q_m | \langle q_p | \frac{z^2}{2m}  {\bf A}_0^2 ( \hat{b}^{\dagger} + \hat{b})^2 | r_m \rangle | r_p \rangle  \\
= & \delta_{q_m, r_m} \left(\frac{z^2}{2m} {\bf A}_0^2 ( 2 r_p \delta_{q_p, r_p} +  \sqrt{r_p + 1} \sqrt{r_p + 2} \delta_{q_p, r_p+2} 
+ \sqrt{r_p} \sqrt{r_p -1} \delta_{q_p, r_p-2})
\right)
\end{align}

In [14]:
    
def compute_diamagnetic_element_p_dot_A(q_m, q_p, r_m, r_p, z_charge, A0, m):
    """ 
    Arguments
    ---------
    q_m : int
        matter bra index

    q_p : int
        photonic bra index

    r_m : int
        matter ket index

    r_p : int
        photon ket index

    z_charge : float
        the effective charge of the matter subsystem

    A0 : float
        the magnitude of the vector potential of the photonic subsystem

    m : float
        the reduced mass of the matter subsystem

    Returns
    -------

    E_diamag : float
        Diamagnetic matrix element 
    """
    fac = z_charge ** 2 * A0 ** 2 / ( 2 * m)

    # must be diagonal in photon space
    E_diamag = 0
    if q_m == r_m:
        if q_p == r_p:
            E_diamag = 2 * fac * (r_p + 1/2)
        
        elif q_p == r_p + 2:
            E_diamag = fac * np.sqrt(r_p + 1) * np.sqrt(r_p + 2)
        elif q_p == r_p - 2:
            E_diamag = fac * np.sqrt(r_p) * np.sqrt(r_p - 1)
        else:
            E_diamag = 0
            
    return E_diamag
    

The function `compute_interaction_matrix_element_p_dot_A(q_m, q_p, r_m, r_p, m, k, z_charge, A0)` computes the matrix element of 

\begin{align}
&-  \frac{z}{m} \langle q_m | \langle q_p |  \hat{p} \cdot  {\bf A}_0 ( \hat{b}^{\dagger} + \hat{b})  | r_m \rangle | r_p \rangle  \\
&= -\frac{z}{m} \langle q_m | \hat{p}    | r_m \rangle  \langle q_p | {\bf A}_0 ( \hat{b}^{\dagger} + \hat{b}) | r_p \rangle \\
&= - i \sqrt{\frac{m \omega_{\rm f}}{2}} \frac{z}{m}  {\bf A}_0  \langle q_m |   \hat{a}^{\dagger} - \hat{a}  | r_m \rangle  \langle q_p |  ( \hat{b}^{\dagger} + \hat{b}) | r_p \rangle \\
&= - i \sqrt{\frac{m \omega_{\rm f}}{2}} \frac{z}{m}  {\bf A}_0  (\sqrt{r_m+1} \delta_{q_m,r_m+1} - \sqrt{r_m} \delta_{q_m, r_m-1} ) ( \sqrt{r_p+1} \delta_{q_p,r_p+1} + \sqrt{r_p} \delta_{q_p, r_p-1}) 
\end{align}
where we have used the definition of the matter momentum operator $\hat{p} = i \sqrt{m \hbar \omega_{\rm f} / 2} (\hat{a}^{\dagger} - \hat{a})$.

In [15]:
def compute_interaction_matrix_element_p_dot_A(q_m, q_p, r_m, r_p, m, k, z_charge, A0):
    """ 
    Arguments
    ---------
    q_m : int
        matter bra index

    q_p : int
        photonic bra index

    r_m : int
        matter ket index

    r_p : int
        photon ket index

    m : float
        the reduced mass of the matter subsystem

    k : float 
        the force constant of the matter subsystem

    z_charge : float
        the effective charge of the matter subsystem

    A0 : float
        the magnitude of the vector potential of the photonic subsystem

    Returns
    -------

    E_coupling : float
        coupling matrix element
    """
    omega_f = np.sqrt( k / m) 
    p0 = 1j * np.sqrt(m * omega_f / 2)
    
    fac = -z_charge * A0 * p0 / m

    q_m, q_p, r_m, r_p, m,
    # matter terms
    if q_m == r_m+1:
        term_1 = np.sqrt(r_m + 1)
    else:
        term_1 = 0
     
    if q_m == r_m-1:
        term_2 = -np.sqrt(r_m)
    else:
        term_2 = 0
    
    # photon terms
    if q_p == r_p+1:
        term_3 = np.sqrt(r_p + 1)
    else:
        term_3 = 0
        
    if q_p == r_p-1:
        term_4 = np.sqrt(r_p)
    else:
        term_4 = 0

    E_coupling = fac * (term_1 + term_2) * (term_3 + term_4)
        
    return E_coupling

The function `compute_interaction_matrix_element_PF(q_m, q_p, r_m, r_p, omega_p, z_charge, A0, k, mu)` will compute 
matrix elements of 
\begin{align}
& -\omega_{\rm cav}\langle q_m | \langle q_p |  \hat{\mu} \cdot {\bf A}_0 ( \hat{b}^{\dagger} + \hat{b}) | r_m \rangle | r_p \rangle \\
=& -\omega_{\rm cav} z \langle q_m | \hat{x} | r_m \rangle \langle q_p |  {\bf A}_0 ( \hat{b}^{\dagger} + \hat{b})  | r_p \rangle \\
=& -\omega_{\rm cav} z \sqrt{\frac{\hbar}{2m \omega_{\rm f}}} {\bf A}_0\langle q_m |( \hat{a}^{\dagger} + \hat{a}) | r_m \rangle \langle q_p |   ( \hat{b}^{\dagger} + \hat{b})  | r_p \rangle \\
=& -\omega_{\rm cav} z \sqrt{\frac{\hbar}{2m \omega_{\rm f}}} {\bf A}_0(\sqrt{r_m+1} \delta_{q_m,r_m+1} + \sqrt{r_m} \delta_{q_m, r_m-1} ) ( \sqrt{r_p+1} \delta_{q_p,r_p+1} + \sqrt{r_p} \delta_{q_p, r_p-1})
\end{align}

In [16]:


def compute_interaction_matrix_element_PF(q_m, q_p, r_m, r_p, omega_p, z_charge, A0, k, m):
    """ 
    Arguments
    ---------
    q_m : int
        matter bra index

    q_p : int
        photonic bra index

    r_m : int
        matter ket index

    r_p : int
        photon ket index

    omega_p : float
        photon frequency
    
    z_charge : float
        the effective charge of the matter subsystem

    A0 : float
        the magnitude of the vector potential of the photonic subsystem

    k : float 
        the force constant of the matter subsystem

    m : float
        the reduced mass of the matter subsystem

    Returns
    -------

    E_coupling : float
        coupling matrix element
    """
    hbar = 1 # plancks constant / 2 * pi in atomic units
    omega_f = np.sqrt( k / m) 
    x0 = np.sqrt( 1 / (2 * m * omega_f))
    
    fac = -omega_p * z_charge * x0 * A0
    
    # matter terms
    if q_m == r_m+1:
        term_1 = np.sqrt(r_m + 1)
    else:
        term_1 = 0
     
    if q_m == r_m-1:
        term_2 = np.sqrt(r_m)
    else:
        term_2 = 0
    
    # photon terms
    if q_p == r_p+1:
        term_3 = np.sqrt(r_p + 1)
    else:
        term_3 = 0
        
    if q_p == r_p-1:
        term_4 = np.sqrt(r_p)
    else:
        term_4 = 0

    E_coupling = fac * (term_1 + term_2) * (term_3 + term_4)
        
    return E_coupling

The function `compute_dipole_self_energy_PF(q_m, q_p, r_m, r_p, omega_p, z_charge, A0, k, mu)` computes the matrix element of 

\begin{align}
&  \frac{\omega_{{\rm cav}}}{\hbar} {\bf A}_0^2 \langle q_m |  \hat{\mu}^2 | r_m \rangle \langle q_p | r_p \rangle  \\
& = \frac{\omega_{{\rm cav}}}{\hbar} {\bf A}_0^2 z^2 \langle q_m |  \hat{x}^2 | r_m \rangle   \\
&= \frac{\omega_{{\rm cav}}}{\hbar} {\bf A}_0^2 z^2  \sqrt{\frac{\hbar}{2m\omega_{\rm f}}}\langle q_m | (\hat{a}^{\dagger} + \hat{a}) | r_m \rangle \delta_{q_p, r_p}\\
&= \frac{\omega_{{\rm cav}}}{\hbar} {\bf A}_0^2 z^2  \frac{\hbar}{2m\omega_{\rm f}} ( 2 r_m \delta_{q_m, r_m} +  \sqrt{r_m + 1} \sqrt{r_m + 2} \delta_{q_m, r_m+2} 
+ \sqrt{r_m} \sqrt{r_m -1} \delta_{q_m, r_m-2}) \delta_{q_p, r_p}\\
\end{align}

In [22]:
def compute_dipole_self_energy_PF(q_m, q_p, r_m, r_p, omega_p, z_charge, A0, k, m):
    """ 
    Arguments
    ---------
    q_m : int
        matter bra index

    q_p : int
        photonic bra index

    r_m : int
        matter ket index

    r_p : int
        photon ket index

    omega_p : float
        photon frequency
    
    z_charge : float
        the effective charge of the matter subsystem

    A0 : float
        the magnitude of the vector potential of the photonic subsystem

    k : float 
        the force constant of the matter subsystem

    m : float
        the reduced mass of the matter subsystem

    Returns
    -------

    E_dse : float
        dipole self energy matrix element
    """
    hbar = 1
    omega_f = np.sqrt( k / m )
    x0 = np.sqrt( 1 / (2 * m * omega_f) )
    fac = omega_p * z_charge ** 2 * x0 ** 2 * A0 ** 2

    # must be diagonal in photon space
    val = 0
    if q_p == r_p:
        if q_m == r_m:
            val = 2 * fac * (r_m + 1/2)
        
        elif q_m == r_m + 2:
            val = fac * np.sqrt(r_m + 1) * np.sqrt(r_m + 2)
        elif q_m == r_m - 2:
            val = fac * np.sqrt(r_m) * np.sqrt(r_m - 1)
        else:
            val = 0
            
    return val
    

The following functions will build the basis used for these matrices, and will build these matrices and diagonalize them, returning both their eigenvectors and eigenvalues.  

In [23]:
def build_basis(photon_dim, matter_dim):
    basis = []
    for i in range(photon_dim):
        for j in range(matter_dim):
            basis.append((j,i))
    return basis


def build_and_diagonalize_p_dot_A(basis, k, mu, omega, z, A0):
    # length of slice of first column gives us the dimension of the Hamiltonian
    dim = len(basis)
    
    # initialize our Hamiltonian
    H_pda = np.zeros((dim,dim), dtype=complex)


    ket_idx = 0
    for ket in basis:

        bra_idx = 0
        
        for bra in basis:
            # matter term
            H_m_element = compute_matter_matrix_element(bra[0], bra[1], ket[0], ket[1], k, mu)
            # photon term
            H_p_element = compute_photon_matrix_element(bra[0], bra[1], ket[0], ket[1], omega)
            # interaction term
            H_i_element = compute_interaction_matrix_element_p_dot_A(bra[0], bra[1], ket[0], ket[1], mu, k, z, A0) 
            # diamagnetic term 
            H_d_element = compute_diamagnetic_element_p_dot_A(bra[0], bra[1], ket[0], ket[1], z, A0, mu)

            H_pda[bra_idx, ket_idx] = H_m_element + H_p_element + H_i_element + H_d_element
            bra_idx = bra_idx + 1
        ket_idx = ket_idx + 1 #ket_idx += 1
    
    # compute eigenvalues and eigenvectors
    vals, vecs = la.eigh(H_pda)
    
    # only return vals
    return H_pda, vals

def build_and_diagonalize_PF(basis, k, mu, omega, z, A0):
    # length of slice of first column gives us the dimension of the Hamiltonian
    dim = len(basis)
    
    # initialize our Hamiltonian
    H_PF = np.zeros((dim,dim), dtype=complex)


    ket_idx = 0
    for ket in basis:

        bra_idx = 0
        
        for bra in basis:

            H_m_element = compute_matter_matrix_element(bra[0], bra[1], ket[0], ket[1], k, mu)

            H_p_element = compute_photon_matrix_element(bra[0], bra[1], ket[0], ket[1], omega)

            H_i_element = compute_interaction_matrix_element_PF(bra[0], bra[1], ket[0], ket[1], omega, z, A0, k, mu)

            H_dse_element = compute_dipole_self_energy_PF(bra[0], bra[1], ket[0], ket[1], omega, z, A0, k, mu)

            H_PF[bra_idx, ket_idx] = H_m_element + H_p_element + H_i_element + H_dse_element
            
            bra_idx = bra_idx + 1
        
        ket_idx = ket_idx + 1 #ket_idx += 1
    
    # compute eigenvalues and eigenvectors
    vals, vecs = la.eigh(H_PF)
    
    # only return vals
    return H_PF, vals





    
        
    


In [28]:
basis_array = build_basis(3, 3)
k_val = 1
mu_val = 1
z_val = 1
omega_p_val = 1
A0_val = 0.5

H_pda, vals_pda = build_and_diagonalize_p_dot_A(basis_array, k_val, mu_val, omega_p_val, z_val, A0_val)
H_PF, vals_PF = build_and_diagonalize_PF(basis_array, k_val, mu_val, omega_p_val, z_val, A0_val)

print(vals_pda)
print(vals_PF)

assert np.allclose(vals_pda, vals_PF)

[1.06183995 1.78587314 2.49414054 2.60211032 3.31684784 3.87315999
 4.01689999 5.34682632 5.87730191]
[1.06183995 1.78587314 2.49414054 2.60211032 3.31684784 3.87315999
 4.01689999 5.34682632 5.87730191]
